# <center>Centro Universitário Facens<br/></center>

## <center>Liga de IA<br/></center>
  
<font size="3"><center>Prof. Renato Moraes Silva</center></font>
<br/>

# <center>Você sobreviveria ao Titanic?</center>

Neste notebook, temos um exemplo de como gerar uma saída que pode ser usada para submeter uma solução para a competição disponível no Kaggle: https://www.kaggle.com/competitions/ai-league-titanic-chalenge

Primeiro, iremos importar as bibliotecas que serão usadas. 

In [8]:
import numpy as np #importa a biblioteca usada para trabalhar com vetores e matrizes
import pandas as pd #importa a biblioteca usada para trabalhar com dataframes (dados em formato de tabela) e análise de dados

import os

Vamos baixar os arquivos

In [16]:
#import urllib.request
from zipfile import ZipFile # biblioteca para arquivos zipados

#url = 'https://www.dropbox.com/s/vhg7heemzfx652f/dados.zip?dl=0'

# especifica o local onde ficarao os arquivos
pathFiles = 'dados/'

# cria uma pasta onde ficarao os arquivos
if not os.path.isdir(pathFiles):
    os.mkdir(pathFiles)

# faz o download do arquivo
#os.system('wget -O %stitanic.zip %s' %(pathFiles,url))

# descompacta os arquivos
z = ZipFile(pathFiles + 'titanic.zip', 'r')
z.extractall(pathFiles)
z.close()

print('Arquivos extraídos com sucesso!')

Arquivos extraídos com sucesso!


Agora, vamos carregar os dados do arquivo.

In [17]:
df_train = pd.read_csv( pathFiles + 'train.csv', sep=',', index_col=None) 
df_test = pd.read_csv( pathFiles + 'test.csv', sep=',', index_col=None) 
df_submissao = pd.read_csv( pathFiles + 'gender_submission.csv', sep=',', index_col=None) 

display(df_train.head(n=3))
display(df_test.head(n=3))
display(df_submissao.head(n=3))

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


,PassengerId,Survived
0,892,0
1,893,1
2,894,0


Eu irei remover todos os atributos, deixando apenas os atributos **Survived**, **SibSp** e **Fare** na base de treino e apenas **SibSp** e **Fare** na base de teste (no teste não há a coluna Survived)  

Essa seleção está sendo feita sem critério algum.  

In [18]:
print(df_train.columns)

df_train = df_train[['Survived','Age','Fare']]
df_test = df_test[['Age','Fare']]

# imprime o dataframe
display(df_train.head(n=5))
display(df_test.head(n=5))

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


,Age,Fare
0,34.5,7.8292
1,47.0,7.0000
2,62.0,9.6875
3,27.0,8.6625
4,22.0,12.2875


Vamos substituir todos os valores nulos por 0.

In [19]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

Iremos jogar os dados para matrizes. A matriz X guardará as amostras, enquanto que a matriz Y guardará as classes.

In [22]:
# dados de treino
X_train = df_train.iloc[:,1:].values
Y_train = df_train.iloc[:,0].values

# dados de teste
X_test = df_test.values

print('\n5 primeiros dados de treino: ')
print(X_train[0:5,:])

print('\n5 primeiras classes de treino: ')
print(Y_train[0:5])

print('\n5 primeiros dados de teste: ')
print(X_test[0:5,:])


5 primeiros dados de treino: 
[[22.      7.25  ]
 [38.     71.2833]
 [26.      7.925 ]
 [35.     53.1   ]
 [35.      8.05  ]]

5 primeiras classes de treino: 
[0 1 1 1 0]

5 primeiros dados de teste: 
[[34.5     7.8292]
 [47.      7.    ]
 [62.      9.6875]
 [27.      8.6625]
 [22.     12.2875]]


Vamos usar os dados de treino para treinar uma classificador chamado de K-vizinhos mais próximos. 


In [23]:
import sklearn as skl
from sklearn import neighbors

# inicia o classificador 
model = skl.neighbors.KNeighborsClassifier(n_neighbors = 3)

# treina o classificador com os dados de treinameto
model.fit(X_train, Y_train)

print(model)

KNeighborsClassifier(n_neighbors=3)


Vamos usar o classificador treinado acima para predizer as classes da base de dados de teste.


In [24]:
Y_pred = model.predict(X_test)

print('5 primeiras classes preditas')
print(Y_pred[0:5])

5 primeiras classes preditas
[0 0 1 1 0]


Vamos preparar o arquivo de submissão, adicionando as classes que fizemos predição no lugar das classes que estão no lugar da variável **df_submissao**. 

In [25]:
df_submissao['Survived'] = Y_pred

Vamos salvar o arquivo que deverá ser enviado ao Kaggle. 

In [26]:
df_submissao.to_csv('dados/minhas_predicoes.csv', index=False)

Agora, submeta o arquivo salvo no Kaggle.